In [1]:
# 참고 사이트 
# https://notebook.community/zzsza/Datascience_School/29.%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C/01.%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%EC%9D%98%20%EA%B8%B0%EC%B4%88
#  유저가 부여한 도서 평점을 회귀 예측하는 AI 모델을 개발해야 합니다.

In [2]:
!unzip '/content/drive/MyDrive/Dacon/open.zip'

Archive:  /content/drive/MyDrive/Dacon/open.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install scikit-surprise
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 49.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095438 sha256=6a42b2940b9aa72b4f0be157f61acd4bb421267171da4be7e59cdbbb568ecb0d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [5]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [6]:
train.head()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


In [7]:
# Surprise 라이브러리용 Reader 및 Dataset 객체 생성
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()

In [8]:
# SVD 모델 훈련
model = SVD()
model.fit(train)

In [13]:
submit = pd.read_csv('./sample_submission.csv')

In [14]:
submit['Book-Rating'] = test.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)

개선


In [10]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [11]:
reader = Reader(rating_scale=(0, 10))
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train = train.build_full_trainset()

In [12]:
# 하이퍼 파라메터 튜닝
from surprise.model_selection import GridSearchCV
param_grid = {
    'n_factors' :[50,100,150],
    'n_epochs' :[20,30,40],
    'lr_all' :[0.002,0.005,0.008],
    'reg_all' :[0.02,0.04,0.06]
}

In [ ]:
from surprise.model_selection import train_test_split
train = pd.read_csv('./train.csv')
train = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
gs = GridSearchCV(SVD,param_grid, cv=5, measures=['rmse'],joblib_verbose=5)
gs.fit(train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   37.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   48.4s remaining:    0.0s
